# Caffe的Python接口学习(1)：生成配置文件

Caffe是C++语言写的，可能很多人不太熟悉，因此想用更简单的脚本语言来实现。Caffe提供Matlab接口和Python接口，这两种语言就非常简单，而且非常容易进行可视化，使得学习更加快速，理解更加深入。

半年前，我在学习Caffe的时候，为了加深理解，因此写下了随笔，有了一系列的Caffe学习文章。半年过去了，很多人问到关于python接口和可视化的一些问题，现在有点空闲时间，就再次写下了一些随笔，大家一起来学习。有些重复的内容，我就不再多讲了，如果大家有兴趣可以移步：

[如何配置Caffe的Python接口？](http://www.cnblogs.com/denny402/p/5088399.html)

[如何将图片转换成LMDB文件？](http://www.cnblogs.com/denny402/p/5082341.html)

[如何计算训练数据的均值文件？](http://www.cnblogs.com/denny402/p/5102328.html)

以上这些操作都是训练之前的预处理操作，不管是用什么接口，都要用到。

首先，我们需要掌握的，就是如何写配置文件，通过下面的代码来学习：

``` python
# -*- coding: utf-8 -*-
from caffe import layers as L, params as P, to_proto

# 保存数据和配置文件的路径
path = '/root/Repository/caffe/examples/mine/python-interface/'
# 训练数据LMDB文件的位置
train_lmdb = path + 'train_db'
# 验证数据LMDB文件的位置
val_lmdb = path + 'val_db'
# 均值文件的位置
mean_file = path + 'mean.binaryproto'
# 生成的训练配置文件保存的位置
train_proto = path + 'train.prototxt'
# 生成的验证配置文件保存的位置
val_proto = path + 'val.prototxt'

# 编写一个函数，用于生成网络
def create_net(lmdb, batch_size, include_acc=False):
    # 创建第一层：数据层。向上传递两类数据：图片数据和对应的标签
    data, label = L.Data(source=lmdb, backend=P.Data.LMDB, batch_size=batch_size, ntop=2,
                         transform_param=dict(crop_size=40, mean_file=mean_file, mirror=True))
    # 创建第二层：卷积层
    conv1 = L.Convolution(data, kernel_size=5, stride=1, num_output=16, pad=2, weight_filter=dict(type='xavier'))
    # 创建激活函数层
    relu1 = L.ReLU(conv1, in_place=True)
    # 创建池化层
    pool1 = L.Pooling(relu1, pool=P.Pooling.MAX, kernel_size=3, stride=2)
    # 以下三行是下一个卷积&池化层
    conv2 = L.Convolution(pool1, kernel_size=3, stride=1, num_output=32, pad=1, weight_filter=dict(type='xavier'))
    relu2 = L.ReLU(conv2, in_place=True)
    pool2 = L.Pooling(relu2, pool=P.Pooling.MAX, kernel_size=3, stride=2)
    # 创建一个全连接层
    fc3 = L.InnerProduct(pool2, num_output=1024, weight_filter=dict(type='xavier'))
    relu3 = L.ReLU(fc3, in_place=True)
    # 创建一个dropout层
    drop3 = L.Dropout(relu3, in_place=True)
    fc4 = L.InnerProduct(drop3, num_output=10, weight_filter=dict(type='xavier'))
    # 创建一个Softmax层
    loss = L.SoftmaxWithLoss(fc4, label)
    
    # 在训练阶段，不需要accuracy层，但是在验证阶段，是需要的
    if include_acc:
        acc = L.Accuracy(fc4, label)
        return to_proto(loss, acc)
    else:
        return to_proto(loss)

def write_net():
    # 将以上的设置写入到prototxt文件
    with open(train_proto, 'w') as f:
        f.write(str(create_net(train_lmdb, batch_size=64)))
        
    # 写入配置文件
    with open(val_proto, 'w') as f:
        f.write(str(create_net(val_lmdb, batch_size=32, include_acc=True)))
        
if __name__ == '__main__':
    write_net()
```

通过上面这个文件的执行，我们就会得到两个配置文件：train.prototxt和val.prototxt，分别用于训练阶段和验证阶段。

这种方式生成配置文件，必须有个前提，就是要先把原始图片转换成LMDB文件才行。如果我们已经把原始图片做成了一个列表清单（txt文件，一行一张图片），则可以不用LMDB格式作为输入数据，可以用ImageData作为数据源输入，代码如下：

``` python
# -*- coding: utf-8 -*-
from caffe import layers as L, params as P, to_proto

# 保存数据和配置文件的路径
path = '/root/Repository/caffe/examples/mine/python-interface/'
# 训练数据文件列表
train_lmdb = path + 'train.txt'
# 验证数据文件列表
val_lmdb = path + 'val.txt'
# 生成的训练配置文件保存的位置
train_proto = path + 'train.prototxt'
# 生成的验证配置文件保存的位置
val_proto = path + 'val.prototxt'

# 编写一个函数，用于生成网络
def create_net(img_list, batch_size, include_acc=False):
    # 创建第一层：数据层。向上传递两类数据：图片数据和对应的标签
    data, label = L.ImageData(source=img_list, batch_size=batch_size, new_width=48, new_height=48, ntop=2,
                         transform_param=dict(crop_size=40, mirror=True))
    # 创建第二层：卷积层
    conv1 = L.Convolution(data, kernel_size=5, stride=1, num_output=16, pad=2, weight_filter=dict(type='xavier'))
    # 创建激活函数层
    relu1 = L.ReLU(conv1, in_place=True)
    # 创建池化层
    pool1 = L.Pooling(relu1, pool=P.Pooling.MAX, kernel_size=3, stride=2)
    # 以下三行是下一个卷积&池化层
    conv2 = L.Convolution(pool1, kernel_size=3, stride=1, num_output=32, pad=1, weight_filter=dict(type='xavier'))
    relu2 = L.ReLU(conv2, in_place=True)
    pool2 = L.Pooling(relu2, pool=P.Pooling.MAX, kernel_size=3, stride=2)
    # 创建一个全连接层
    fc3 = L.InnerProduct(pool2, num_output=1024, weight_filter=dict(type='xavier'))
    relu3 = L.ReLU(fc3, in_place=True)
    # 创建一个dropout层
    drop3 = L.Dropout(relu3, in_place=True)
    fc4 = L.InnerProduct(drop3, num_output=10, weight_filter=dict(type='xavier'))
    # 创建一个Softmax层
    loss = L.SoftmaxWithLoss(fc4, label)
    
    # 在训练阶段，不需要accuracy层，但是在验证阶段，是需要的
    if include_acc:
        acc = L.Accuracy(fc4, label)
        return to_proto(loss, acc)
    else:
        return to_proto(loss)

def write_net():
    # 将以上的设置写入到prototxt文件
    with open(train_proto, 'w') as f:
        f.write(str(create_net(train_lmdb, batch_size=64)))
        
    # 写入配置文件
    with open(val_proto, 'w') as f:
        f.write(str(create_net(val_lmdb, batch_size=32, include_acc=True)))
        
if __name__ == '__main__':
    write_net()
```

即第一层由原来的Data类型，变成了ImageData类型，不需要LMDB文件和均值文件，但需要一个txt文件。